In [2]:

import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from openpyxl import load_workbook


def imports():
    orders = pd.read_excel("FILE.xlsx", sheet_name="OPEN_ORDERS")
    orders = pd.DataFrame(orders)
    orders = orders.rename(columns={"Item\nNumber":"Item Number"})
    orders['Sales Reporting Code 03'] = orders['Sales Reporting Code 03'].astype(int)
    orders['KEY'] = orders['BU\nHeader'].astype(str)+"_"+orders['Sales Reporting Code 03'].astype(str)
    
    cat = pd.read_excel("FILE.xlsx", sheet_name="CAT")
    cat['KEY'] = cat['RBU'].astype(str)+"_"+cat['SalesCode'].astype(str)
    
    
    con = pd.read_excel("FILE.xlsx",sheet_name="CONVERSION")
    
    date = pd.read_excel("File.xlsx",sheet_name="ORDERDATE")
    
    
    ship = pd.read_excel("FILE.xlsx",sheet_name="SHIPMENT")
    ship = ship[["Branch/Plant","Sold To","Sold To Name",
         "Ship To","2nd Item Number","Unit Price",
         "Quantity Shipped","Sls Cd3","Invoice Date","Or Ty"]]
    
    ship = ship[(ship['Branch/Plant']==164)|(ship['Branch/Plant']==166)|(ship['Branch/Plant']==168)]
    ship = ship[(ship['Or Ty']=="SO")|(ship['Or Ty']=="SE")|(ship['Or Ty']=="SC")]
    ship['Sls Cd3'] = ship['Sls Cd3'].replace(' ', np.nan)
    ship['Sls Cd3'] = ship['Sls Cd3'].astype(float).astype(pd.Int64Dtype())
    #ship['Sls Cd3'] = ship['Sls Cd3'].astype(int)
    ship["KEY1"] = ship['Ship To'].astype(str)+"_"+ship['Sls Cd3'].astype(str)
    
    acc = pd.read_excel("FILE.xlsx",sheet_name="ACCOUNT")
    acc = acc.fillna(0)
    acc['SHIPTO_ACC'] = acc['SHIPTO_ACC'].astype(int)
    acc['SalesCode'] = acc['SalesCode'].astype(int)
    acc['RBU'] = acc['RBU'].astype(int)
    acc['KEY1'] = acc['SHIPTO_ACC'].astype(str) + "_" + acc['SalesCode'].astype(str)
    
    
    
    return orders, con, cat, date,ship,acc

def concat(orders,con, cat,date,ship,acc):
    orders = pd.merge(orders, con[["Item Number","Conversion Factor"]], on="Item Number", how='left')
    orders['Conversion Factor'] = orders['Conversion Factor'].fillna(1)
    orders['OpenO'] = orders['Conversion Factor'].astype(int) * orders['Qty\nOrder/\nTransaction'].astype(int)
    orders = pd.merge(orders,cat[["KEY","CAT","GROUP"]], on="KEY", how='left')
    orders = orders[orders['Order\nDate']==date['Date'].iloc[0]]
    orders = orders[["GROUP","CAT","OpenO","Item Number","BU\nHeader","Branch/\nPlant","Total Amount"]]
    
    
    ship = pd.merge(ship,acc[["KEY1","RBU"]],on='KEY1',how="left")
    NonRegistedAcc = ship[ship['RBU'].isna()]
    ship = ship.rename(columns={"2nd Item Number":"Item Number"})
    ship = pd.merge(ship, con[["Item Number","Conversion Factor"]], on="Item Number", how='left')
    ship['Conversion Factor'] = ship['Conversion Factor'].fillna(1)
    ship = ship[~ship['KEY1'].isin(NonRegistedAcc['KEY1'])]
    ship['RBU'] = ship['RBU'].astype(int)
    ship['KEY'] = ship['RBU'].astype(str)+"_"+ship['Sls Cd3'].astype(str)
    ship['Shipped'] = ship['Quantity Shipped'].astype(float)* ship['Conversion Factor'].astype(float)
    ship = pd.merge(ship,cat[["KEY","CAT","GROUP"]], on="KEY", how='left')
    ship['Total_Sales'] = ship['Unit Price'].astype(float)*ship['Quantity Shipped'].astype(float)


    return orders, ship,NonRegistedAcc
   
    
orders, con, cat, date, ship,acc = imports()
orders,ship,NonRegistedAcc = concat(orders,con,cat,date, ship,acc)

excel_file = 'Results.xlsx'
book = load_workbook(excel_file)

# Create a Pandas ExcelWriter using the loaded workbook
with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    writer.book = book
    
    # Check if the 'OpenOrders' sheet exists, and if so, clear its content
    if 'OpenOrders' in writer.book.sheetnames:
        sheet = writer.book['OpenOrders']
        sheet.delete_rows(sheet.min_row + 1, sheet.max_row)
    
    # Write the DataFrame to the 'OpenOrders' sheet
    orders.to_excel(writer, sheet_name='OpenOrders', index=False)
    
    # Check if the 'Shipped' sheet exists, and if so, clear its content
    if 'Shipped' in writer.book.sheetnames:
        sheet = writer.book['Shipped']
        sheet.delete_rows(sheet.min_row + 1, sheet.max_row)
    
    ship.to_excel(writer, sheet_name='Shipped', index=False)
    
    # Check if the 'Update' sheet exists, and if so, clear its content
    if 'Update' in writer.book.sheetnames:
        sheet = writer.book['Update']
        sheet.delete_rows(sheet.min_row + 1, sheet.max_row)
    
    NonRegistedAcc.to_excel(writer, sheet_name='Update', index=False)

# Save the changes to the Excel file
book.save(excel_file)

C:\Users\gwalal\AppData\Local\Temp\ipykernel_10972\564840536.py:79: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
